#Environment Setup & Repair
#Downgrade google colab to compatible version

1. Downgrade google colab to compatible version
2. Install langchain-aws to connect with bedrock
3. Streamlite for web interface
4. pyngrok for URL(public Tunnel)
5. boto3 SDK for python







In [ ]:

import os
import subprocess
import sys

print(" Repairing environment... (This takes ~45 seconds)")

# 1. Force uninstall the incompatible Numpy 2.x (Colab default)
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "numpy"])

# 2. Install the correct Numpy 1.x that works with Streamlit/AWS
subprocess.run([sys.executable, "-m", "pip", "install", "numpy<2.0"])

# 3. Install AI dependencies (LangChain, Bedrock, Streamlit, Ngrok)
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "langchain", "langchain-aws", "boto3", "streamlit", "pyngrok"])

print("Installation Complete")

 Repairing environment... (This takes ~45 seconds)
Installation Complete


##Main block for AI application
#Technical Features
1. Cross-Region Inference
2. AWS inference profile for routing
3. Secure tunneling usig Ngrok to expose local stremline server



In [ ]:
import os
import time
import subprocess
import threading
from google.colab import userdata
from pyngrok import ngrok

#AWS access
try:
    AWS_ACCESS_KEY = userdata.get('AWS_ACCESS_KEY_ID')
    AWS_SECRET_KEY = userdata.get('AWS_SECRET_ACCESS_KEY')
    NGROK_TOKEN = userdata.get('NGROK_AUTH_TOKEN')

    # Region selection
    AWS_REGION = "eu-central-1"

except Exception as e:
    print(f"❌ Secret Error: {e}")

# APP Generation
app_code = """
import streamlit as st
import boto3
import os
from langchain_aws import ChatBedrock
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

st.set_page_config(page_title="🤖Legacy Code Summarizer", layout="wide")

# 1. Get Credentials
aws_access_key = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

# FORCE UPDATE: Hardcode fallback to eu-central-1
aws_region = os.environ.get('AWS_DEFAULT_REGION', 'eu-central-1')

# 2. Bedrock Logic with Inference Profile
def get_llm():
    client = boto3.client(
        service_name="bedrock-runtime",
        region_name=aws_region,
        aws_access_key_id=aws_access_key,
        aws_secret_access_key=aws_secret_key
    )

    # UPDATE: Using the EU Inference Profile ID
    return ChatBedrock(
        client=client,
        model_id="eu.anthropic.claude-3-5-sonnet-20240620-v1:0",
        model_kwargs={"temperature": 0.0, "max_tokens": 4000}
    )

def analyze_code(code, lang):
    llm = get_llm()
    prompt = ChatPromptTemplate.from_template("Explain this {lang} code logic: {code}")
    chain = prompt | llm | StrOutputParser()
    return chain.invoke({"lang": lang, "code": code})

st.title(" AI Legacy Code Summarizer")
st.markdown(f"**Region:** `{aws_region}` | **Model:** `Sonnet EU`")

col1, col2 = st.columns(2)
with col1:
    lang = st.selectbox("Language", ["Python", "Java", "COBOL"])
    code = st.text_area("Code", height=300)
    btn = st.button("Analyze")

with col2:
    if btn and code:
        try:
            res = analyze_code(code, lang)
            st.success("Analysis Complete")
            st.write(res)
        except Exception as e:
            st.error(f"Error: {e}")
"""

# Write the file to the Colab environment
with open("app.py", "w") as f:
    f.write(app_code)

# --- PART C: Launch with Explicit Environment ---
ngrok.kill()
subprocess.run(["pkill", "streamlit"])

if NGROK_TOKEN:
    ngrok.set_auth_token(NGROK_TOKEN)
else:
    print("⚠️ WARNING: NGROK_AUTH_TOKEN not found. The link might not work.")

# We create a specific environment dictionary to pass to the app
app_env = os.environ.copy()
app_env["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY
app_env["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET_KEY
app_env["AWS_DEFAULT_REGION"] = AWS_REGION  # <--- This passes "eu-central-1"

# Run Streamlit in background (Headless mode)
cmd = ["streamlit", "run", "app.py", "--server.address=0.0.0.0", "--server.headless=true"]
process = subprocess.Popen(cmd, env=app_env, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

time.sleep(3)
try:
    public_url = ngrok.connect(8501).public_url
    print(f"\n🚀 RELAUNCHED IN EU-CENTRAL-1: {public_url}")
    print("(If you see a warning page, click 'Visit Site')")
except Exception as e:
    print(f"Tunnel failed: {e}")

# Keep cell running
import asyncio
try:
    asyncio.get_event_loop().run_forever()
except:
    pass

Tunnel failed: ngrok client exception, API returned 502: {"error_code":103,"status_code":502,"msg":"failed to start tunnel","details":{"err":"failed to start tunnel: The endpoint 'https://aidyn-unshaped-semiarticulately.ngrok-free.dev' is already online. Either\n1. stop your existing endpoint first, or\n2. start both endpoints with `--pooling-enabled` to load balance between them.\r\n\r\nERR_NGROK_334\r\n"}}

